#Define command line tasks for pre-ingest transformation

* [DM-2191](https://jira.lsstcorp.org/browse/DM-2191), John Swinbank, 6 SPs.

Issue description:

> DM-1903 provided a command line task which would transform a src catalogue into calibrated form. Here, we build on that to provide command line tasks for all source catalogues which will need to be ingested; will include at least `deepCoadd_src`, `goodSeeingCoadd_src`, `chiSquaredCoadd_src`.
